In [1]:
import numpy as np
import pickle
import os

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import style

from PIL import Image, ImageDraw

radarData_path =  'data/072819_zl_onNotOn/f_data-2019-07-28_22-11-01.258054_zl_onNotOn_rnn/f_data.p'
videoData_path = 'data/072819_zl_onNotOn/v_data-2019-07-28_22-10-32.249041_zl_onNotOn_rnn/cam1'
mergedImg_path = 'E:/figures/zl_onNotOn'
radarImg_path = 'radar_test'

radar_data = list(pickle.load(open(radarData_path, 'rb')).items())
radar_data.sort(key=lambda x: x[0])  # sort by timestamp
videoData_list = os.listdir(videoData_path)
videoData_timestamps = list(map(lambda x: float(x.strip('.jpg')), videoData_list))

style.use('fivethirtyeight')
color = 'rgb(255, 255, 255)'
for timestamp, data in radar_data:
    i = radar_data.index((timestamp, data))
    print('Processing ' + str(i + 1) + ' of ' + str(len(radar_data)))

    closest_video_timestamp = min(videoData_timestamps,
                                  key=lambda x: abs(x - timestamp))
    closest_video_path = os.path.join(videoData_path, str(closest_video_timestamp) + '.jpg')
    closest_video_img = Image.open(closest_video_path)

    # plot the radar scatter
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim((-0.3, 0.3))
    ax.set_ylim((-0.3, 0.3))
    ax.set_zlim((-0.3, 0.3))

    # ax.scatter(data['x'], data['y'], data['z'], c=data['doppler'], marker='D')

    ax.scatter(data['x'], data['y'], data['z'], s=10 * data['doppler'], marker='D')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.savefig(os.path.join(radarImg_path, str(timestamp) + '.jpg'))
    plt.clf()

    radar_img = Image.open(os.path.join(radarImg_path, str(timestamp) + '.jpg'))

    images = [closest_video_img, radar_img]

    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset, 0))
        x_offset += im.size[0]

    timestamp_difference = abs(float(timestamp) - float(closest_video_timestamp))
    draw = ImageDraw.Draw(new_im)

    # draw the timestamp difference on the image
    (x, y) = (50, 70)
    message = "Timestamp Difference, abs(rt-vt): " + str(timestamp_difference)
    draw.text((x,y), message, fill=color)

    # draw the timestamp
    (x, y) = (50, 50)
    message = "Timestamp: " + str(timestamp)
    draw.text((x, y), message, fill=color)

    new_im.save(os.path.join(mergedImg_path, str(timestamp) + '.jpg'))


Processing 1 of 3953


c:\users\s-vec\appdata\local\programs\python\python37\lib\site-packages\matplotlib\collections.py:874: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor


FileNotFoundError: [Errno 2] No such file or directory: 'radar_test\\1564366262.1876683.jpg'